In [55]:
from google.cloud import storage
import pandas as pd 
from google.cloud import bigquery 

In [56]:
PROJECT_ID = "isi-group-m2-dsia"
BUCKET_NAME = "m2dsia-attoisse-mohamed-data"

In [57]:
client = storage.Client(project=PROJECT_ID)
bucket = client.get_bucket(BUCKET_NAME)

In [58]:
blobs = bucket.list_blobs(prefix="input/")
for blob in blobs:
    print(blob.name)

input/
input/transactions.csv


In [59]:
def upload_blob(data, destination_blob_name):
    blob = bucket.blob(destination_blob_name)
    
    if isinstance(data, str):
        blob.upload_from_string(data, content_type='text/csv')
    else:
        blob.upload_from_string(data, content_type='application/octet-stream')
    
    print(f"Fichier uploadé vers {destination_blob_name}.")

In [60]:
def delete_file(filename):
    try:
        blob = bucket.blob(filename)
        if blob.exists():
            blob.delete()  
            print(f"Le fichier {filename} a été supprimé avec succès.")
        else:
            print(f"Le fichier {filename} n'existe pas dans le bucket.")
    except Exception as e:
        print(f"Une erreur s'est produite lors de la suppression du fichier {filename}: {e}")

In [65]:
delete_file('input/transactions.csv')

Le fichier input/transactions.csv a été supprimé avec succès.


In [36]:
def clean_data(df):
    required_columns = ['transaction_id', 'product_name', 'category', 'price', 'quantity', 'date']
    if df[required_columns].isnull().any().any():
        raise ValueError("Champs obligatoires manquants. Impossible de nettoyer les données.")
    
    mode_customer_name = df['customer_name'].mode()[0]  
    df['customer_name'] = df['customer_name'].fillna(mode_customer_name)
    
    mode_customer_email = df['customer_email'].mode()[0]  
    df['customer_email'] = df['customer_email'].fillna(mode_customer_email)

    df['transaction_id'] = df['transaction_id'].astype(int)  
    df['product_name'] = df['product_name'].astype(str)   
    df['category'] = df['category'].astype(str)   
    df['price'] = df['price'].astype(float)  
    df['quantity'] = df['quantity'].astype(int)  
    df['date'] = pd.to_datetime(df['date']).dt.date   

    df = df.drop_duplicates()
    
    return df

In [37]:
def validate_data(df):
    required_columns = ['transaction_id', 'product_name', 'category', 'price', 'quantity', 'date']
    if not all(column in df.columns for column in required_columns):
        return False

    if df[required_columns].isnull().any().any():
        return False

    try:
        df['transaction_id'] = df['transaction_id'].astype(int)  
        df['product_name'] = df['product_name'].astype(str)   
        df['category'] = df['category'].astype(str)   
        df['price'] = df['price'].astype(float)  
        df['quantity'] = df['quantity'].astype(int)  
        df['date'] = pd.to_datetime(df['date']).dt.date  
    except ValueError:
        return False
    
    return True

In [45]:
def process_file(file_path):
    try:
        df = pd.read_csv(file_path)
        if not validate_data(df):
            raise ValueError("Données invalides. Champs obligatoires manquants ou types incorrects.")
        
        df_cleaned = clean_data(df)
        
        destination_path = f"clean/{file_path.split('/')[-1]}"
        upload_blob(df_cleaned.to_csv(index=False), destination_path)
        print(f"Fichier {file_path} traité et déplacé vers clean/.")
        
        return df_cleaned
    except Exception as e:
        destination_path = f"error/{file_path.split('/')[-1]}"
        
        with open(file_path, 'rb') as file:
            file_content = file.read()
            upload_blob(file_content, destination_path)
        
        print(f"Erreur lors du traitement du fichier {file_path}. Déplacé vers error/. Erreur : {e}")

In [46]:
def add_to_bigquery(df, table_id):
    """
    Ajoute les données du DataFrame à une table BigQuery.
    
    :param df: DataFrame contenant les données à ajouter.
    :param table_id: ID de la table BigQuery (ex: "project_id.dataset_id.table_id").
    """
    client = bigquery.Client()
    job = client.load_table_from_dataframe(df, table_id)
    job.result()  
    
    print(f"Données ajoutées à la table BigQuery : {table_id}")

In [47]:
def main(file_path, table_id):
    
    try:
        df_process = process_file(file_path)
        add_to_bigquery(df_process, table_id)
        destination_path = f"done/{file_path.split('/')[-1]}" 
        upload_blob(df_process.to_csv(index=False), destination_path)
        print(f"Fichier {file_path} traité, ajouté à la table BigQuery, et déplacé vers done/.")
        
    except Exception as e:
        destination_path = f"error/{file_path.split('/')[-1]}"  
        with open(file_path, 'rb') as file:
            file_content = file.read()
            upload_blob(file_content, destination_path)
        print(f"Erreur lors du traitement du fichier {file_path}. Déplacé vers error/. Erreur : {e}")

In [50]:
file_to_process = "files/errors/error1.csv"  
table_id = f"{PROJECT_ID}.dataset_attoisse_mohamed.transactions"
main(file_to_process,table_id)

Fichier uploadé vers error/error1.csv.
Erreur lors du traitement du fichier files/errors/error1.csv. Déplacé vers error/. Erreur : Données invalides. Champs obligatoires manquants ou types incorrects.
Fichier uploadé vers error/error1.csv.
Erreur lors du traitement du fichier files/errors/error1.csv. Déplacé vers error/. Erreur : 'NoneType' object has no attribute 'columns'
